# Memory Based collaborative filtering 


In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import json
import warnings
import sys
sys.path.append('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data_collection/notiAPI')

from auth import NotiServerAuth
from api import NotiServerRequest

from dataclasses import dataclass, field
from typing import Optional, List
from urllib.parse import urlparse, parse_qs
from sklearn.metrics.pairwise import cosine_similarity

from pprint import pprint
warnings.filterwarnings(action='ignore')

## 데이터 로드

In [ ]:
# 최근 10일치 데이터, spec + 상품 id 추출
df = pd.read_csv('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data/spec.csv') 
df = df[['user', 'item']]
df

## 데이터 전처리 

In [ ]:
# 인터렉션 횟수가 5회 이상인 아이템들의 데이터 추출
interaction_counts = df['item'].value_counts()

# 5회 이상인 유저가 2290 으로 매우 적음
item_with_5_or_more_interactions = interaction_counts[interaction_counts >= 5].index
df = df[df['item'].isin(item_with_5_or_more_interactions)]
df

---

## Item based Collaborative Filtering (IBCF)

User-Item matrix를 만들고 코사인 유사도를 이용해 유사한 아이템을 추천 해보겠습니다.

In [ ]:
user_item_matrix = df.pivot_table(index='item', columns='user', aggfunc=len, fill_value=0)
user_item_matrix

In [ ]:
# 아이템 사이 코사인 유사도 계산

item_similarity = cosine_similarity(user_item_matrix) 
print('item_similarity shape: ', item_similarity.shape)
item_similarity

In [ ]:
def IBCF(df: pd.DataFrame,
         item: str,
         topn: Optional[int]=None) -> pd.DataFrame:
    topn=11 if topn is None else topn+1

    if item in df['item'].values:
        # 아이템-사용자 행렬 생성
        item_user_matrix = df.pivot_table(index='item', columns='user', aggfunc=len, fill_value=0)
        # 아이템 간 코사인 유사도 계산
        item_similarity_df = pd.DataFrame(cosine_similarity(item_user_matrix), index=item_user_matrix.index.astype(str), columns=item_user_matrix.index.astype(str))
        # 유사도가 높은 상위 n개 아이템 반환
        sim_item_df = item_similarity_df[item].sort_values(ascending=False).reset_index().rename(columns={'index':'item',item:'similarity'})
        print('입력한 아이템 id: ',item)
        return(sim_item_df[1:topn])
    else:
        print('아이템 id 다시 확인해주세요')

In [ ]:
# 'Apple 2023 맥북 프로 16 M3 Pro 12코어 CPU 18코어 GPU, 영문자판, 스페이스 블랙, 512GB, 18GB, 영문'

result = IBCF(df,'010c861e9e040c29e1132eb02d0841bec065e863941d2e609bf039fdc50a57b8', 5)
result

In [ ]:
# 입력 아이템
res = NotiServerRequest.bulk_product_info(ids=['010c861e9e040c29e1132eb02d0841bec065e863941d2e609bf039fdc50a57b8'], pivs=[])
print('입력 아이템 :', res.json().get('data', {}).get('products', [])[0]['title'].split(',')[0])


# 아이디 리스트
item_ids = result['item'].tolist()

# 각 아이디에 대해 제품 정보를 가져와서 타이틀을 출력
for item_id in item_ids:
    res = NotiServerRequest.bulk_product_info(ids=[item_id], pivs=[])
    product_info = res.json().get('data', {}).get('products', [])
    if product_info:
        title = product_info[0]['title'].split(',')[0]
        # print("아이템 ID:", item_id)
        print("추천 :", title)
    else:
        # print("아이템 ID:", item_id)
        print("해당 아이템에 대한 정보를 찾을 수 없습니다.")


---